# 1.  Import packages

In [2]:
import time
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
pd.set_option("max_colwidth", 200)

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.chrome.options import Options
from icecream import ic
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
import os

chrome_options = Options()
chrome_options.headless = True
chrome_drive = './chromedriver'
os.environ['webdrive.chrome.drive'] = chrome_drive

from pandarallel import pandarallel
num_workers =  72
pandarallel.initialize(nb_workers = num_workers)
# pandarallel.initialize()

INFO: Pandarallel will run on 72 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# 2. Get countries and its links of news

## 2.1 Get countries main pages

In [2]:
url = 'https://www.ohchr.org/EN/Countries/Pages/HumanRightsintheWorld.aspx'
coverpage = requests.get(url).content
soup = BeautifulSoup(coverpage, 'html.parser')
items = soup.find_all('div', class_="link-item" )

In [3]:
df = pd.DataFrame([[item.a.text, 'https://www.ohchr.org' + item.a['href']] for item in items], columns  = ['country','country_url'])
df

,country,country_url
0,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx
1,Benin,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BJIndex.aspx
2,Botswana,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BWIndex.aspx
3,Burkina Faso,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BFIndex.aspx
4,Burundi,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BIIndex.aspx
...,...,...
191,United Arab Emirates,https://www.ohchr.org/EN/countries/MENARegion/Pages/AEIndex.aspx
192,Yemen,https://www.ohchr.org/EN/countries/MENARegion/Pages/YEIndex.aspx
193,Cook Islands,https://www.ohchr.org/EN/countries/AsiaRegion/Pages/CKIndex.aspx
194,Holy See,https://www.ohchr.org/EN/countries/ENACARegion/Pages/VAIndex.aspx


## 2.2. Get the links of news from each country's main page

recent news and archieve news (over 3 years old）

In [4]:
def extract_url(url):
    coverpage = requests.get(url).content
    soup = BeautifulSoup(coverpage, 'html.parser')

    link_list =  soup.find_all(text=re.compile(r'.*(Recent news|over 3 years old).*')) 

    if len(link_list)<2:
        print('\terror %s'%url)
    hrefs = []
    for link in link_list: 
        href = link.parent['href']
        if href[0] == '/':
            href = 'https://www.ohchr.org' + str(href)
        hrefs.append(href)
    return hrefs

In [5]:
df[['news_search_link']] = None

In [6]:
# batch_size = 36
# for i in range(0,df.shape[0],batch_size):
#     print(i ,end = '\r')
#     df.loc[i:i+batch_size-1, 'news_search_link']=\
#     df.loc[i:i+batch_size-1, 'country_url'].parallel_apply(extract_url).values

In [6]:
df.loc[:, 'news_search_link']=\
df.loc[:, 'country_url'].parallel_apply(extract_url).values

	error https://www.ohchr.org/EN/countries/ENACARegion/Pages/VAIndex.aspx
	error https://www.ohchr.org/EN/countries/AsiaRegion/Pages/CKIndex.aspx


Error means some countries don't have the links to recent news and archieve news. We shall remove those entries

In [7]:
df[df['news_search_link'].str.len()<2]

,country,country_url,news_search_link
193,Cook Islands,https://www.ohchr.org/EN/countries/AsiaRegion/Pages/CKIndex.aspx,[]
194,Holy See,https://www.ohchr.org/EN/countries/ENACARegion/Pages/VAIndex.aspx,[]


In [8]:
print('Before removing empty links', df.shape)

df = df[df.news_search_link.str.len()>0]
print('After removing', df.shape)

Before removing empty links (196, 3)
After removing (194, 3)


In [9]:
df = df.explode('news_search_link').reset_index(drop=True)
df

,country,country_url,news_search_link
0,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
1,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,http://newsarchive.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
2,Benin,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BJIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BJ
3,Benin,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BJIndex.aspx,http://newsarchive.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BJ
4,Botswana,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BWIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BW
...,...,...,...
383,United Arab Emirates,https://www.ohchr.org/EN/countries/MENARegion/Pages/AEIndex.aspx,http://newsarchive.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=DZ
384,Yemen,https://www.ohchr.org/EN/countries/MENARegion/Pages/YEIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=YE
385,Yemen,https://www.ohchr.org/EN/countries/MENARegion/Pages/YEIndex.aspx,http://newsarchive.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=YE
386,Occupied Palestinian Territory,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS


In [10]:
df.to_csv('ohchr_raw.csv', index = False)

# 3. Get each record from search tables

Load saved raw dataset that contains news links per country

In [2]:
df = pd.read_csv('ohchr_raw.csv',header= [0])
df.head()

,country,country_url,news_search_link
0,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
1,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,http://newsarchive.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
2,Benin,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BJIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BJ
3,Benin,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BJIndex.aspx,http://newsarchive.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BJ
4,Botswana,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BWIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BW


In [3]:
df1=  df.copy()
# df1[['rows']]= None
# df1.head()

In [4]:
def get_rows_from_table(url, human_involve= True):
    drive = webdriver.Chrome(options=chrome_options, executable_path=chrome_drive)
    drive.get(url)

    rows=[]
    
    repeat_num = 0

    while True:
        next_soup = BeautifulSoup(drive.page_source, 'html.parser')
        next_rows_find = next_soup.find_all('table', cellspacing = '0', border = '1')
        next_rows = next_rows_find[0].find_all('tr', align="left")[0:-1]
        rows += [str(i) for i in next_rows]
        
        try:            
#             next_find = drive.find_element_by_xpath("//*[contains(text(), 'Next -->')]")
            WebDriverWait(drive, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, 'Next -->'))).click()

        except NoSuchElementException: # "No more pages"
            break
        except ElementNotInteractableException:

            repeat_num +=1
            if human_involve:
                print('Try to solve: %s\trepeat times: %s \n'%(url[-2:],repeat_num))
                drive.maximize_window()
                time.sleep(20)
                drive.set_window_size(1024, 800)
            else: 
                print('ElementNotInteractableException: %s\n'%url)
                break

    drive.quit()
    
    if repeat_num:
        rows = list(set(rows)) 
    
    return rows

Warning: the next line of code takes a long time because it needs to click next pages of tables to get all the rows.

In [13]:
# batch_size =  8
# pandarallel.initialize(nb_workers = num_workers)

# batch_size = 16
# for i in range(160,df1.shape[0],batch_size):
#     print(i ,end = '\r')
    
#     df1.loc[i:i+batch_size-1, 'rows']=\
#     df1.loc[i:i+batch_size-1, 'news_search_link'].parallel_apply(get_rows_from_table).values

In [5]:
# pandarallel.initialize(nb_workers = 128)
# df1.loc[:, 'rows']= df1.loc[:, 'news_search_link'].parallel_apply(get_rows_from_table).values
df1[['rows']] = df1.news_search_link.parallel_map(get_rows_from_table).values

In [6]:
df1.rows.str.len().describe()

count    388.000000
mean      54.041237
std       45.092192
min        1.000000
25%       23.000000
50%       43.000000
75%       70.000000
max      359.000000
Name: rows, dtype: float64

In [7]:
df1[df1.rows.str.len()<3]

,country,country_url,news_search_link,rows
209,Nauru,https://www.ohchr.org/EN/countries/AsiaRegion/Pages/NRIndex.aspx,http://newsarchive.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=NR,"[<tr align=""left"" style=""color:Black;background-color:#D3DFEE;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_Place..."
225,Samoa,https://www.ohchr.org/EN/countries/AsiaRegion/Pages/WSIndex.aspx,http://newsarchive.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=WS,"[<tr align=""left"" style=""color:Black;background-color:#D3DFEE;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_Place..."


In [8]:
df1 = df1.explode('rows').reset_index(drop=True)
df1

,country,country_url,news_search_link,rows
0,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#D3DFEE;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH..."
1,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#FFFFFF;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH..."
2,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#D3DFEE;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH..."
3,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#FFFFFF;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH..."
4,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#D3DFEE;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH..."
...,...,...,...,...
20963,Occupied Palestinian Territory,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,http://newsarchive.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS,"<tr align=""left"" style=""color:Black;background-color:#D3DFEE;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH..."
20964,Occupied Palestinian Territory,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,http://newsarchive.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS,"<tr align=""left"" style=""color:Black;background-color:#FFFFFF;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH..."
20965,Occupied Palestinian Territory,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,http://newsarchive.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS,"<tr align=""left"" style=""color:Black;background-color:#D3DFEE;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH..."
20966,Occupied Palestinian Territory,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,http://newsarchive.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS,"<tr align=""left"" style=""color:Black;background-color:#FFFFFF;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH..."


In [11]:
df1.to_csv('ohchr_rows.csv', index = False)

# 4 get html per record

In [2]:
df1 = pd.read_csv('ohchr_rows.csv', header = [0])

In [3]:
df2  = df1.copy()
df2[['Title', 'Date', 'News_Type','Multi_countries',  'Mandate', 'subject','story_link', 'story_html']]= None
# df2.head()

Check if the row contains English links and extract its content. If not, ignore it.

In [4]:
def get_row_html(row):
    
    soup = BeautifulSoup(row, 'html.parser')
    link =  soup.find_all('a', text='E')[0]
    
    if 'href' not in link.attrs:
        return pd.Series(['']*8)
    
    else:
        link = link['href']
        if link[0:11] == 'DisplayNews':
                  link = 'https://www.ohchr.org/en/NewsEvents/Pages/'+ link
                  link_page = requests.get(link).content   
                    
        for item in soup.findAll('li'):
            item = item.replace_with(item.text + ',')
            
        text =  re.sub("^\s+|\s+$", "", soup.text, flags=re.UNICODE)
        text = re.sub("\n+", "\n", text)
        text = re.sub(",\n", "\n", text)
        text_list = text.split('\n')
            
    return  pd.Series(text_list[0:6] + [link] + [str(link_page)])

In [6]:
df2['Title', 'Date', 'News_Type','Multi_countries', 'Mandate', 'subject','story_link', 'story_html']\
= df2.rows.parallel_map(get_row_html).values

In [20]:
df2

,country,country_url,news_search_link,rows,Title,Date,News_Type,Multi_countries,Mandate,subject,story_link,story_html
0,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#D3DFEE;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH...","Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan",3/12/2020,Press Releases,"Fiji,San Marino,Iran (Islamic Republic of),Angola,Kazakhstan",Human Rights Council,Human Rights Council,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,"b'\r\n<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 Transitional//EN"" ""http://www.w3.org/TR/html4/loose.dtd"">\n<html __expr-val-dir=""ltr"" dir=""ltr"" class=""ms-isBot"" lang=""en-US"">\r\n <head><meta..."
1,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#FFFFFF;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH...",Angola’s human rights record to be reviewed by Universal Periodic Review,11/4/2019,Press Releases,Angola,"Human Rights Council,Universal Periodic Review","Universal Periodic Review,Human Rights Council",https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,"b'\r\n<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 Transitional//EN"" ""http://www.w3.org/TR/html4/loose.dtd"">\n<html __expr-val-dir=""ltr"" dir=""ltr"" class=""ms-isBot"" lang=""en-US"">\r\n <head><meta..."
2,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#D3DFEE;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH...",Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,10/30/2019,Press Releases,"Italy,El Salvador,Gambia,Bolivia (Plurinational State of),Fiji,San Marino,Kazakhstan,Angola,Iran (Islamic Republic of),Madagascar,Iraq,Slovenia,Egypt,Bosnia and Herzegovina","Universal Periodic Review,Human Rights Council","Universal Periodic Review,Human Rights Council",https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,"b'\r\n<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 Transitional//EN"" ""http://www.w3.org/TR/html4/loose.dtd"">\n<html __expr-val-dir=""ltr"" dir=""ltr"" class=""ms-isBot"" lang=""en-US"">\r\n <head><meta..."
3,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#FFFFFF;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH...",Human Rights Committee adopts Its annual report and closes Its one-hundred and twenty-fifth session,3/29/2019,Press Releases,"Angola,Estonia,Eritrea,Niger,Saint Vincent and the Grenadines,Viet Nam",Human Rights Committee,E,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24430&LangID=E,"b'\r\n<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 Transitional//EN"" ""http://www.w3.org/TR/html4/loose.dtd"">\n<html __expr-val-dir=""ltr"" dir=""ltr"" class=""ms-isBot"" lang=""en-US"">\r\n <head><meta..."
4,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#D3DFEE;bor

In [24]:
df2 = df2[df2.story_link!='']
df2.shape

(19714, 12)

In [26]:
df2.to_csv('ohchr_eng_html.csv', index = False)

# 5.  get plain text from html per record

In [2]:
df3 = pd.read_csv('ohchr_eng_html.csv', header=[0])

In [3]:
# df3 = df2.copy()
df3[['text']] = None
df3.head()

,country,country_url,news_search_link,rows,Title,Date,News_Type,Multi_countries,Mandate,subject,story_link,story_html,text
0,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#D3DFEE;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH...","Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan",3/12/2020,Press Releases,"Fiji,San Marino,Iran (Islamic Republic of),Angola,Kazakhstan",Human Rights Council,Human Rights Council,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,"b'\r\n<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 Transitional//EN"" ""http://www.w3.org/TR/html4/loose.dtd"">\n<html __expr-val-dir=""ltr"" dir=""ltr"" class=""ms-isBot"" lang=""en-US"">\r\n <head><meta...",None
1,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#FFFFFF;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH...",Angola’s human rights record to be reviewed by Universal Periodic Review,11/4/2019,Press Releases,Angola,"Human Rights Council,Universal Periodic Review","Universal Periodic Review,Human Rights Council",https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,"b'\r\n<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 Transitional//EN"" ""http://www.w3.org/TR/html4/loose.dtd"">\n<html __expr-val-dir=""ltr"" dir=""ltr"" class=""ms-isBot"" lang=""en-US"">\r\n <head><meta...",None
2,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#D3DFEE;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH...",Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,10/30/2019,Press Releases,"Italy,El Salvador,Gambia,Bolivia (Plurinational State of),Fiji,San Marino,Kazakhstan,Angola,Iran (Islamic Republic of),Madagascar,Iraq,Slovenia,Egypt,Bosnia and Herzegovina","Universal Periodic Review,Human Rights Council","Universal Periodic Review,Human Rights Council",https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,"b'\r\n<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 Transitional//EN"" ""http://www.w3.org/TR/html4/loose.dtd"">\n<html __expr-val-dir=""ltr"" dir=""ltr"" class=""ms-isBot"" lang=""en-US"">\r\n <head><meta...",None
3,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;background-color:#FFFFFF;border-color:White;border-style:None;font-family:verdana;font-size:1em;"" valign=""top"">\n<td style=""width:30%;"">\n<span id=""ctl00_PlaceH...",Human Rights Committee adopts Its annual report and closes Its one-hundred and twenty-fifth session,3/29/2019,Press Releases,"Angola,Estonia,Eritrea,Niger,Saint Vincent and the Grenadines,Viet Nam",Human Rights Committee,E,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24430&LangID=E,"b'\r\n<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 Transitional//EN"" ""http://www.w3.org/TR/html4/loose.dtd"">\n<html __expr-val-dir=""ltr"" dir=""ltr"" class=""ms-isBot"" lang=""en-US"">\r\n <head><meta...",None
4,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"<tr align=""left"" style=""color:Black;bac

In [14]:
def get_plain_text(link_page):
    link_soup = BeautifulSoup(link_page, 'html.parser')
    full_story_find = link_soup.find_all('span', class_='lblnewsfulltext')
    if len(full_story_find) == 0:
        full_story = ''
    elif len(full_story_find)>=1:
        full_story = full_story_find[0].get_text(' ')         
    return full_story

In [ ]:
def get_plain_text(link_page):
    link_soup = BeautifulSoup(link_page, 'html.parser')
    full_story_find = link_soup.find_all('span', class_='lblnewsfulltext')
    if len(full_story_find) == 0:
        full_story = ''
    elif len(full_story_find)>=1:
        full_story = full_story_find[0].get_text(' ')         
    return full_story

In [15]:
df3.text = df3.story_html.parallel_map(get_plain_text).values

In [16]:
df3.to_csv('ohchr_html_n_text.csv', index = False)

In [35]:
# df3.text.to_csv('ohchr_only_text.csv', index = False)

In [6]:
del df3

In [7]:
from newspaper import Article
def get_news_text(html):
    a = Article('')
    a.set_html(html)
    a.parse()
    text= a.text
    return text

In [8]:
df = pd.read_csv('ohchr_html_n_text.csv',header=0)
df = df[df.text.str.len()>150].reset_index(drop=True)
# df = df.drop(columns=['rows','story_html'])
# df.text = df.text.apply(lambda x: re.sub("^\s+|\s+$", "", x, flags=re.UNICODE))

In [9]:
pandarallel.initialize(progress_bar=True)
df[['news']] = df['story_html'].parallel_apply(get_news_text).values

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [20]:
df = df.drop(columns=['rows','story_html'])
df.text = df.text.apply(lambda x: re.sub("^\s+|\s+$", "", x, flags=re.UNICODE))
df.news = df.news.apply(lambda x: re.sub("^\s+|\s+$", "", x, flags=re.UNICODE))

In [11]:
def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")"))
    return TEXT

In [21]:
import unicodedata

df.text = df.text.apply(lambda x: x.replace('\\n','\n'))
df.text = df.text.apply(lambda x: unicodetoascii(x))
df.text = df.text.apply(lambda x: unicodedata.normalize("NFKD", x))
df.text = df.text.apply(lambda x: re.sub(r"http\S+", "", x))
df.text = df.text.apply(lambda x: re.sub("\S+@\S+(?:\.\S+)+",'',x))
df.text = df.text.apply(lambda x: re.sub('\+( |\d)+',' ',x))

df.text = df.text.apply(lambda x: re.sub("^\s+|\s+$", "", x, flags=re.UNICODE))

# df.news = df.news.apply(lambda x: x.replace('\\n','\n'))
# df.news = df.news.apply(lambda x: unicodetoascii(x))
# df.news = df.news.apply(lambda x: unicodedata.normalize("NFKD", x))
# df.news = df.news.apply(lambda x: re.sub(r"http\S+", "", x))
# df.news = df.news.apply(lambda x: re.sub("\S+@\S+(?:\.\S+)+",'',x))
# df.news = df.news.apply(lambda x: re.sub('\+( |\d)+',' ',x))

# df.news = df.news.apply(lambda x: re.sub("^\s+|\s+$", "", x, flags=re.UNICODE))

In [13]:
df.news = df.news.apply(lambda x: re.sub("\n\n+", "\n\n", x, flags=re.UNICODE))

In [25]:
df = df[df.news.str.len()>200].reset_index(drop=True)

In [31]:
# df = df.drop(columns='text')
# df
# df = df.rename(columns ={'news':'text'})
df = df.drop(columns='news_search_link')


,country,country_url,news_search_link,Title,Date,News_Type,Multi_countries,Mandate,subject,story_link,text
0,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan",3/12/2020,Press Releases,"Fiji,San Marino,Iran (Islamic Republic of),Angola,Kazakhstan",Human Rights Council,Human Rights Council,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,"Human Rights Council\n\n12 March 2020\n\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan.\n\nNazhat S..."
1,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,Angola’s human rights record to be reviewed by Universal Periodic Review,11/4/2019,Press Releases,Angola,"Human Rights Council,Universal Periodic Review","Universal Periodic Review,Human Rights Council",https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,Portuguese version\n\nGENEVA (4 November 2019) - Angola's human rights record will be examined by the UN Human Rights Council's Universal Periodic Review (UPR) Working Group for the third time on ...
2,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,10/30/2019,Press Releases,"Italy,El Salvador,Gambia,Bolivia (Plurinational State of),Fiji,San Marino,Kazakhstan,Angola,Iran (Islamic Republic of),Madagascar,Iraq,Slovenia,Egypt,Bosnia and Herzegovina","Universal Periodic Review,Human Rights Council","Universal Periodic Review,Human Rights Council",https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,Working Group to Review the Human Rights Records in 14 States\n\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council\'s Universal Periodic Review (UPR) Working Group w...
3,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,Human Rights Committee adopts Its annual report and closes Its one-hundred and twenty-fifth session,3/29/2019,Press Releases,"Angola,Estonia,Eritrea,Niger,Saint Vincent and the Grenadines,Viet Nam",Human Rights Committee,E,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24430&LangID=E,"ROUNDUP\n\nAdopts Concluding Observations and Recommendations on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam\n\nGENEVA (29 March 2019) - The Human Rights Commit..."
4,Angola,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO,"UN Human Rights Committee publishes findings on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam",3/28/2019,Press Releases,"Angola,Eritrea,Estonia,Niger,Saint Vincent and the Grenadines,Viet Nam",Human Rights Committee,E,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24420&LangID=E,"GENEVA (28 March 2019) - The UN Human Rights Committee has published its findings on the civil and political rights record of countries it examined during its latest session: Angola, Eritrea, Esto..."
...,...,...,...,...,...,...,...,...,...,...,...
9189,Occupied Palestinian Territory,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS,"Press briefing notes on South Sudan, Ethiopia, United States, Palestine and Thailand / South East Asia",5/2/2014,Press Briefing Notes,"Ethiopia,South Sudan,United States of America,Occupied Palestinian Territory,

In [50]:
# df = df.rename(columns={'Title':'title', 'Date':'date'})
df = df.rename(columns={'story_link':'url'})
df = df[['title','date','text', 'country','url', 'country_url','news_search_link']]
df

,title,date,text,country,url,country_url,news_search_link
0,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan",3/12/2020,"Human Rights Council\n\n12 March 2020\n\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan.\n\nNazhat S...",Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
1,Angola’s human rights record to be reviewed by Universal Periodic Review,11/4/2019,Portuguese version\n\nGENEVA (4 November 2019) - Angola's human rights record will be examined by the UN Human Rights Council's Universal Periodic Review (UPR) Working Group for the third time on ...,Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
2,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,10/30/2019,Working Group to Review the Human Rights Records in 14 States\n\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council\'s Universal Periodic Review (UPR) Working Group w...,Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
3,Human Rights Committee adopts Its annual report and closes Its one-hundred and twenty-fifth session,3/29/2019,"ROUNDUP\n\nAdopts Concluding Observations and Recommendations on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam\n\nGENEVA (29 March 2019) - The Human Rights Commit...",Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24430&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
4,"UN Human Rights Committee publishes findings on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam",3/28/2019,"GENEVA (28 March 2019) - The UN Human Rights Committee has published its findings on the civil and political rights record of countries it examined during its latest session: Angola, Eritrea, Esto...",Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24420&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
...,...,...,...,...,...,...,...
9189,"Press briefing notes on South Sudan, Ethiopia, United States, Palestine and Thailand / South East Asia",5/2/2014,Spokesperson for the UN High Commissioner for Human Rights: Rupert Colville\n\nLocation: Geneva\n\nDate: 2 May 2014\n\nSubject: 1) South Sudan\n\n2) Ethiopia\n\n3) United States\n\n4) Palestine\n\...,Occupied Palestinian Territory,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14556&LangID=E,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS
9190,Hebron: Israeli settlers must be stopped from taking over Al-Rajabi House – UN Special Rapporteur,4/15/2014,"GENEVA (15 April 2014) - The United Nations Special Rapporteur on the situation of human rights in the Palestinian Territories occupied since 1967, Richard Falk, today urged Israel to prevent sett...",Occupied Palestinian Territory,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14512&LangID=E,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS
9191,Human Rights Council holds General debate on the Human Rights Situation in 

In [51]:
df.to_csv('ohchr.csv', index= False)

In [2]:
df=pd.read_csv('ohchr.csv', header=0)
df

,title,date,text,country,url,country_url,news_search_link
0,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan",3/12/2020,"Human Rights Council\n\n12 March 2020\n\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan.\n\nNazhat S...",Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
1,Angola’s human rights record to be reviewed by Universal Periodic Review,11/4/2019,Portuguese version\n\nGENEVA (4 November 2019) - Angola's human rights record will be examined by the UN Human Rights Council's Universal Periodic Review (UPR) Working Group for the third time on ...,Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
2,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,10/30/2019,Working Group to Review the Human Rights Records in 14 States\n\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council\'s Universal Periodic Review (UPR) Working Group w...,Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
3,Human Rights Committee adopts Its annual report and closes Its one-hundred and twenty-fifth session,3/29/2019,"ROUNDUP\n\nAdopts Concluding Observations and Recommendations on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam\n\nGENEVA (29 March 2019) - The Human Rights Commit...",Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24430&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
4,"UN Human Rights Committee publishes findings on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam",3/28/2019,"GENEVA (28 March 2019) - The UN Human Rights Committee has published its findings on the civil and political rights record of countries it examined during its latest session: Angola, Eritrea, Esto...",Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24420&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
...,...,...,...,...,...,...,...
9189,"Press briefing notes on South Sudan, Ethiopia, United States, Palestine and Thailand / South East Asia",5/2/2014,Spokesperson for the UN High Commissioner for Human Rights: Rupert Colville\n\nLocation: Geneva\n\nDate: 2 May 2014\n\nSubject: 1) South Sudan\n\n2) Ethiopia\n\n3) United States\n\n4) Palestine\n\...,Occupied Palestinian Territory,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14556&LangID=E,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS
9190,Hebron: Israeli settlers must be stopped from taking over Al-Rajabi House – UN Special Rapporteur,4/15/2014,"GENEVA (15 April 2014) - The United Nations Special Rapporteur on the situation of human rights in the Palestinian Territories occupied since 1967, Richard Falk, today urged Israel to prevent sett...",Occupied Palestinian Territory,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14512&LangID=E,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS
9191,Human Rights Council holds General debate on the Human Rights Situation in 

In [5]:
df.drop(columns='date')

,title,text,country,url,country_url,news_search_link
0,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan","Human Rights Council\n\n12 March 2020\n\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan.\n\nNazhat S...",Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
1,Angola’s human rights record to be reviewed by Universal Periodic Review,Portuguese version\n\nGENEVA (4 November 2019) - Angola's human rights record will be examined by the UN Human Rights Council's Universal Periodic Review (UPR) Working Group for the third time on ...,Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
2,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,Working Group to Review the Human Rights Records in 14 States\n\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council\'s Universal Periodic Review (UPR) Working Group w...,Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
3,Human Rights Committee adopts Its annual report and closes Its one-hundred and twenty-fifth session,"ROUNDUP\n\nAdopts Concluding Observations and Recommendations on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam\n\nGENEVA (29 March 2019) - The Human Rights Commit...",Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24430&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
4,"UN Human Rights Committee publishes findings on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam","GENEVA (28 March 2019) - The UN Human Rights Committee has published its findings on the civil and political rights record of countries it examined during its latest session: Angola, Eritrea, Esto...",Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24420&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
...,...,...,...,...,...,...
9189,"Press briefing notes on South Sudan, Ethiopia, United States, Palestine and Thailand / South East Asia",Spokesperson for the UN High Commissioner for Human Rights: Rupert Colville\n\nLocation: Geneva\n\nDate: 2 May 2014\n\nSubject: 1) South Sudan\n\n2) Ethiopia\n\n3) United States\n\n4) Palestine\n\...,Occupied Palestinian Territory,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14556&LangID=E,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS
9190,Hebron: Israeli settlers must be stopped from taking over Al-Rajabi House – UN Special Rapporteur,"GENEVA (15 April 2014) - The United Nations Special Rapporteur on the situation of human rights in the Palestinian Territories occupied since 1967, Richard Falk, today urged Israel to prevent sett...",Occupied Palestinian Territory,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14512&LangID=E,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS
9191,Human Rights Council holds General debate on the Human Rights Situation in Palestine and other occupied Arab Territoires,Human Rights Council\n\nMIDDAY/AF

In [9]:
df_sample = df.sample(n=50).reset_index(drop=True)
for i,j in zip(df_sample.story_link, df_sample.text):
    print(i)
    print(j)
    print('\n')

https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=22892&LangID=E
Human  Rights Committee 22 March 2018The Human Rights  Committee today completed the review of the third periodic report of Lebanon  during which it examined measures taken to implement the provisions of the  International Covenant on Civil and Political Rights.
  
  The review of  Lebanon, which had started on 15 March, could not be completed as scheduled due  to a strike called for by the United Nations staff at the United Nations Office  at Geneva on 16 March.  The review resumed on 22 March with the  participation of Lebanon by a video conference from Beirut, and in the presence  of the representatives of the Permanent Mission of Lebanon to the United  Nations Office at Geneva.
  
  Presenting the  report, Salim Baddoura, Permanent Representative of Lebanon to the United  Nations Office at Geneva, informed the Committee of the adoption of a new  electoral law which recognized the right to vote of the L

In [18]:
print(df.text[1])

\n      Portuguese version\n      GENEVA  (4 November 2019) \xe2\x80\x93 Angola\xe2\x80\x99s  human rights record will be examined by the UN Human Rights Council\xe2\x80\x99s Universal  Periodic Review (UPR) Working Group for the third time on Thursday, 7 November 2019  in a meeting that will be \n      webcast live. Angola is one of the 14 States to be  reviewed by the UPR Working Group during its upcoming session taking place from  4 to 15 November.  Angola\xe2\x80\x99s \n      first and second  UPR reviews took place in February 2010 and October 2014, respectively.The documents on which the reviews are  based are: 1) national report - information provided by the State under review;  2) information contained in the reports of independent human rights experts and  groups, known as the Special Procedures, human rights treaty bodies, and other  UN entities; 3) information provided by other stakeholders including  national human rights institutions, regional organizations and civil socie

In [22]:
df.to_csv('ohchr_final.csv', index = False)

In [44]:
df1 = pd.read_csv('ohchr_final.csv', header=0)

In [45]:
df1 = df[['Title','Date','story_link','text']]
df1.columns=['title','date','url','text']
df1

,title,date,url,text
0,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan",3/12/2020,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,"Human Rights Council\n12 March 2020\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan. \nNazhat Sh..."
1,Angola’s human rights record to be reviewed by Universal Periodic Review,11/4/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,\n \nPortuguese version\n\n \nGENEVA (4 November 2019)\n \xe2\x80\x93 Angola\xe2\x80\x99s human rights record will be examined by the UN Human Rights Council\xe2\x80\x99s Universal Pe...
2,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,10/30/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,Working Group to Review the Human Rights Records in 14 States\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council\'s Universal Periodic Review (UPR) Working Group wil...
3,Human Rights Committee adopts Its annual report and closes Its one-hundred and twenty-fifth session,3/29/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24430&LangID=E,"ROUNDUP\nAdopts Concluding Observations and Recommendations on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam\n \nGENEVA (29 March 2019) - The Human Rights Comm..."
4,"UN Human Rights Committee publishes findings on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam",3/28/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24420&LangID=E,"GENEVA (28 March 2019) \xe2\x80\x94 The UN Human Rights Committee has published its findings on the civil and political rights record of countries it examined during its latest session: Angola,..."
...,...,...,...,...
19375,"Press briefing notes on South Sudan, Ethiopia, United States, Palestine and Thailand / South East Asia",5/2/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14556&LangID=E,\n\n\n\nSpokesperson for the UN High Commissioner for Human Rights\n: Rupert Colville\nLocation:\n Geneva\nDate\n: 2 May 2014 \nSubject:\n 1) South Sudan\n 2) Ethiopia \n ...
19376,Hebron: Israeli settlers must be stopped from taking over Al-Rajabi House – UN Special Rapporteur,4/15/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14512&LangID=E,"\n\n\n\nGENEVA (15 April 2014) \xe2\x80\x93 The United Nations Special Rapporteur on the situation of human rights in the Palestinian Territories occupied since 1967, Richard Falk, today urged Isr..."
19377,Human Rights Council holds General debate on the Human Rights Situation in Palestine and other occupied Arab Territoires,3/24/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14432&LangID=E,\n\n\n\nHuman Rights Council\nMIDDAY/AFTERNOON \n\n\n24 March 2014\n\n\nHolds General Debate on Vienna Declaration and Programme of Action\nThe Human Rights Council in its midday/afternoon meeting...
19378,Human Rights Council holds Interactive dialogue with the Special Rapporteur on the Situation of Human Rights in Palestine,3/24/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14431&LangID=E,"\n\n\n\nHuman Rights Council\nMORNING \n\n\n24 March 2014\n\n\nThe Human Rights Council this morning held an interactive dialogue with Richard Falk, the Special Rapporteur on the situation of huma..."


In [33]:
df1 = df1.reset_index(drop=True)
df1

,title,date,url,text
0,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan",3/12/2020,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,"Human Rights Council\n12 March 2020\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan.Â \nNazhat S..."
1,Angola’s human rights record to be reviewed by Universal Periodic Review,11/4/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,\n \nPortuguese version\n\n \nGENEVA (4 November 2019)\n â Angolaâs human rights record will be examined by the UN Human Rights Councilâs Universal Periodic Review (UPR) Working...
2,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,10/30/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,Working Group to Review the Human Rights Records in 14 States\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council's Universal Periodic Review (UPR) Working Group will...
3,Human Rights Committee adopts Its annual report and closes Its one-hundred and twenty-fifth session,3/29/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24430&LangID=E,"ROUNDUP\nAdopts Concluding Observations and Recommendations on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam\n \nGENEVA (29 March 2019) - The Human Rights Comm..."
4,"UN Human Rights Committee publishes findings on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam",3/28/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24420&LangID=E,"GENEVA (28 March 2019) â The UN Human Rights Committee has published its findings on the civil and political rights record of countries it examined during its latest session: Angola, Eritrea,..."
...,...,...,...,...
9190,"Press briefing notes on South Sudan, Ethiopia, United States, Palestine and Thailand / South East Asia",5/2/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14556&LangID=E,\n\n\n\nSpokesperson for the UN High Commissioner for Human Rights\n: Rupert Colville\nLocation:\nÂ Â Geneva\nDate\n: 2 May 2014 \nSubject:\nÂ Â 1) South Sudan\nÂ Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â ...
9191,Hebron: Israeli settlers must be stopped from taking over Al-Rajabi House – UN Special Rapporteur,4/15/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14512&LangID=E,"\n\n\n\nGENEVA (15 April 2014) â The United Nations Special Rapporteur on the situation of human rights in the Palestinian Territories occupied since 1967, Richard Falk, today urged Israel to pr..."
9192,Human Rights Council holds General debate on the Human Rights Situation in Palestine and other occupied Arab Territoires,3/24/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14432&LangID=E,\n\n\n\nHuman Rights Council\nMIDDAY/AFTERNOONÂ \n\n\n24 March 2014\n\n\nHolds General Debate on Vienna Declaration and Programme of Action\nThe Human Rights Council in its midday/afternoon meetin...
9193,Human Rights Council holds Interactive dialogue with the Special Rapporteur on the Situation of Human Rights in Palestine,3/24/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14431&LangID=E,"\n\n\n\nHuman Rights Council\nMORNINGÂ \n\n\n24 March 2014\n\n\nThe Human Rights Council this morning held an interactive dialogue with Richard Falk, the Special Rapporteur on the situation of hum..."


In [41]:
df1.text = df1.text.apply(lambda x: x.encode('latin-1').decode('unicode_escape'))
df1

/home/yibo/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: invalid escape sequence '\ '
  """Entry point for launching an IPython kernel.


,title,date,url,text
0,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan",3/12/2020,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,"Human Rights Council\n12 March 2020\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan.Â \nNazhat S..."
1,Angola’s human rights record to be reviewed by Universal Periodic Review,11/4/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,\n \nPortuguese version\n\n \nGENEVA (4 November 2019)\n â Angolaâs human rights record will be examined by the UN Human Rights Councilâs Universal Periodic Review (UPR) Working...
2,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,10/30/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,Working Group to Review the Human Rights Records in 14 States\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council's Universal Periodic Review (UPR) Working Group will...
3,Human Rights Committee adopts Its annual report and closes Its one-hundred and twenty-fifth session,3/29/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24430&LangID=E,"ROUNDUP\nAdopts Concluding Observations and Recommendations on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam\n \nGENEVA (29 March 2019) - The Human Rights Comm..."
4,"UN Human Rights Committee publishes findings on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam",3/28/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24420&LangID=E,"GENEVA (28 March 2019) â The UN Human Rights Committee has published its findings on the civil and political rights record of countries it examined during its latest session: Angola, Eritrea,..."
...,...,...,...,...
9190,"Press briefing notes on South Sudan, Ethiopia, United States, Palestine and Thailand / South East Asia",5/2/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14556&LangID=E,\n\n\n\nSpokesperson for the UN High Commissioner for Human Rights\n: Rupert Colville\nLocation:\nÂ Â Geneva\nDate\n: 2 May 2014 \nSubject:\nÂ Â 1) South Sudan\nÂ Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â ...
9191,Hebron: Israeli settlers must be stopped from taking over Al-Rajabi House – UN Special Rapporteur,4/15/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14512&LangID=E,"\n\n\n\nGENEVA (15 April 2014) â The United Nations Special Rapporteur on the situation of human rights in the Palestinian Territories occupied since 1967, Richard Falk, today urged Israel to pr..."
9192,Human Rights Council holds General debate on the Human Rights Situation in Palestine and other occupied Arab Territoires,3/24/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14432&LangID=E,\n\n\n\nHuman Rights Council\nMIDDAY/AFTERNOONÂ \n\n\n24 March 2014\n\n\nHolds General Debate on Vienna Declaration and Programme of Action\nThe Human Rights Council in its midday/afternoon meetin...
9193,Human Rights Council holds Interactive dialogue with the Special Rapporteur on the Situation of Human Rights in Palestine,3/24/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14431&LangID=E,"\n\n\n\nHuman Rights Council\nMORNINGÂ \n\n\n24 March 2014\n\n\nThe Human Rights Council this morning held an interactive dialogue with Richard Falk, the Special Rapporteur on the situation of hum..."


In [53]:
print(df1.text[1])

\n      
Portuguese version
\n      
GENEVA  (4 November 2019)
 \xe2\x80\x93 Angola\xe2\x80\x99s  human rights record will be examined by the UN Human Rights Council\xe2\x80\x99s Universal  Periodic Review (UPR) Working Group for the third time on Thursday, 7 November 2019  in a meeting that will be \n      
webcast live
. 
Angola is one of the 14 States to be  reviewed by the UPR Working Group during its upcoming session taking place from  4 to 15 November.  Angola\xe2\x80\x99s \n      
first and second  UPR reviews
 took place in February 2010 and October 2014, respectively.
The documents on which the reviews are  based are: 1) national report - information provided by the State under review;  2) information contained in the reports of independent human rights experts and  groups, known as the Special Procedures, human rights treaty bodies, and other  UN entities; 3) information provided by other stakeholders including  national human rights institutions, regional organizations and c

In [48]:
import codecs
codecs.decode(df1.text[1], 'unicode_escape')

'\n      \nPortuguese version\n\n      \nGENEVA  (4 November 2019)\n â\x80\x93 Angolaâ\x80\x99s  human rights record will be examined by the UN Human Rights Councilâ\x80\x99s Universal  Periodic Review (UPR) Working Group for the third time on Thursday, 7 November 2019  in a meeting that will be \n      \nwebcast live\n. \nAngola is one of the 14 States to be  reviewed by the UPR Working Group during its upcoming session taking place from  4 to 15 November.Â\xa0 Angolaâ\x80\x99s \n      \nfirst and second  UPR reviews\n took place in February 2010 and October 2014, respectively.\nThe documents on which the reviews are  based are: 1) national report - information provided by the State under review;  2) information contained in the reports of independent human rights experts and  groups, known as the Special Procedures, human rights treaty bodies, and other  UN entities; 3) information provided by other stakeholders including  national human rights institutions, regional organizations an

In [47]:
df1.text[1].replace('\\\\','\\')

'\\n      \nPortuguese version\n\\n      \nGENEVA  (4 November 2019)\n \\xe2\\x80\\x93 Angola\\xe2\\x80\\x99s  human rights record will be examined by the UN Human Rights Council\\xe2\\x80\\x99s Universal  Periodic Review (UPR) Working Group for the third time on Thursday, 7 November 2019  in a meeting that will be \\n      \nwebcast live\n. \nAngola is one of the 14 States to be  reviewed by the UPR Working Group during its upcoming session taking place from  4 to 15 November.\xa0 Angola\\xe2\\x80\\x99s \\n      \nfirst and second  UPR reviews\n took place in February 2010 and October 2014, respectively.\nThe documents on which the reviews are  based are: 1) national report - information provided by the State under review;  2) information contained in the reports of independent human rights experts and  groups, known as the Special Procedures, human rights treaty bodies, and other  UN entities; 3) information provided by other stakeholders including  national human rights institutions

In [25]:
df1.to_csv('ohchr.csv', index = False)

,title,date,url,text
0,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan",3/12/2020,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,"Human Rights Council\n12 March 2020\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan. \nNazhat Sh..."
1,Angola’s human rights record to be reviewed by Universal Periodic Review,11/4/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,\n \nPortuguese version\n\n \nGENEVA (4 November 2019)\n \xe2\x80\x93 Angola\xe2\x80\x99s human rights record will be examined by the UN Human Rights Council\xe2\x80\x99s Universal Pe...
2,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,10/30/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,Working Group to Review the Human Rights Records in 14 States\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council\'s Universal Periodic Review (UPR) Working Group wil...
3,Human Rights Committee adopts Its annual report and closes Its one-hundred and twenty-fifth session,3/29/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24430&LangID=E,"ROUNDUP\nAdopts Concluding Observations and Recommendations on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam\n \nGENEVA (29 March 2019) - The Human Rights Comm..."
4,"UN Human Rights Committee publishes findings on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam",3/28/2019,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24420&LangID=E,"GENEVA (28 March 2019) \xe2\x80\x94 The UN Human Rights Committee has published its findings on the civil and political rights record of countries it examined during its latest session: Angola,..."
...,...,...,...,...
19375,"Press briefing notes on South Sudan, Ethiopia, United States, Palestine and Thailand / South East Asia",5/2/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14556&LangID=E,\n\n\n\nSpokesperson for the UN High Commissioner for Human Rights\n: Rupert Colville\nLocation:\n Geneva\nDate\n: 2 May 2014 \nSubject:\n 1) South Sudan\n 2) Ethiopia \n ...
19376,Hebron: Israeli settlers must be stopped from taking over Al-Rajabi House – UN Special Rapporteur,4/15/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14512&LangID=E,"\n\n\n\nGENEVA (15 April 2014) \xe2\x80\x93 The United Nations Special Rapporteur on the situation of human rights in the Palestinian Territories occupied since 1967, Richard Falk, today urged Isr..."
19377,Human Rights Council holds General debate on the Human Rights Situation in Palestine and other occupied Arab Territoires,3/24/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14432&LangID=E,\n\n\n\nHuman Rights Council\nMIDDAY/AFTERNOON \n\n\n24 March 2014\n\n\nHolds General Debate on Vienna Declaration and Programme of Action\nThe Human Rights Council in its midday/afternoon meeting...
19378,Human Rights Council holds Interactive dialogue with the Special Rapporteur on the Situation of Human Rights in Palestine,3/24/2014,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14431&LangID=E,"\n\n\n\nHuman Rights Council\nMORNING \n\n\n24 March 2014\n\n\nThe Human Rights Council this morning held an interactive dialogue with Richard Falk, the Special Rapporteur on the situation of huma..."


In [7]:
df1 =pd.read_csv('ohchr.csv', header=0)
df1

,title,text,country,url,country_url,news_search_link
0,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan","Human Rights Council\n\n12 March 2020\n\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan.\n\nNazhat S...",Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
1,Angola’s human rights record to be reviewed by Universal Periodic Review,Portuguese version\n\nGENEVA (4 November 2019) - Angola's human rights record will be examined by the UN Human Rights Council's Universal Periodic Review (UPR) Working Group for the third time on ...,Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
2,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,Working Group to Review the Human Rights Records in 14 States\n\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council\'s Universal Periodic Review (UPR) Working Group w...,Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
3,Human Rights Committee adopts Its annual report and closes Its one-hundred and twenty-fifth session,"ROUNDUP\n\nAdopts Concluding Observations and Recommendations on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam\n\nGENEVA (29 March 2019) - The Human Rights Commit...",Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24430&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
4,"UN Human Rights Committee publishes findings on Angola, Eritrea, Estonia, Niger, Saint Vincent and the Grenadines, and Viet Nam","GENEVA (28 March 2019) - The UN Human Rights Committee has published its findings on the civil and political rights record of countries it examined during its latest session: Angola, Eritrea, Esto...",Angola,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=24420&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/AOIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=AO
...,...,...,...,...,...,...
9189,"Press briefing notes on South Sudan, Ethiopia, United States, Palestine and Thailand / South East Asia",Spokesperson for the UN High Commissioner for Human Rights: Rupert Colville\n\nLocation: Geneva\n\nDate: 2 May 2014\n\nSubject: 1) South Sudan\n\n2) Ethiopia\n\n3) United States\n\n4) Palestine\n\...,Occupied Palestinian Territory,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14556&LangID=E,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS
9190,Hebron: Israeli settlers must be stopped from taking over Al-Rajabi House – UN Special Rapporteur,"GENEVA (15 April 2014) - The United Nations Special Rapporteur on the situation of human rights in the Palestinian Territories occupied since 1967, Richard Falk, today urged Israel to prevent sett...",Occupied Palestinian Territory,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14512&LangID=E,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS
9191,Human Rights Council holds General debate on the Human Rights Situation in Palestine and other occupied Arab Territoires,Human Rights Council\n\nMIDDAY/AF

In [8]:
df1.text.str.len().describe()

count     9194.000000
mean     11110.418751
std      12778.448149
min        449.000000
25%       2897.750000
50%       4981.000000
75%      14392.250000
max      99288.000000
Name: text, dtype: float64

In [9]:
import unicodedata
def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")"))
    return TEXT


# Convert coding
df1.text = df1.text.apply(lambda x: unicodetoascii(x))
df1.text = df1.text.apply(lambda x: unicodedata.normalize("NFKD", x))

# email
df1.text = df1.text.apply(lambda x: re.sub("\S+@\S+(?:\.\S+)+",'',x))

# telphone
df1.text = df1.text.apply(lambda x: re.sub('\(\+( |-|\d)+\)( |-|\d)+',' ',x))
df1.text = df1.text.apply(lambda x: re.sub('\+( |-|\d)+',' ',x))

# noise
df1.text =\
df1.text.apply(lambda x: re.sub('\n(ad|advertisement|tweet):?\n', "", x, flags=re.IGNORECASE))

# urls
df1.text = df1.text.apply(lambda x: re.sub(r"http\S+", "", x))

# delete too many \n
df1.text = df1.text.apply(lambda x: re.sub('\n\n+', "\n\n", x, flags=re.IGNORECASE))

# head and tails
df1.text = df1.text.apply(lambda x: re.sub("^\s+|\s+$", "", x, flags=re.UNICODE)) 


In [10]:
df1.text.str.len().describe()

count     9194.000000
mean     11110.418534
std      12778.448257
min        449.000000
25%       2897.750000
50%       4981.000000
75%      14392.250000
max      99288.000000
Name: text, dtype: float64

In [11]:
df1.shape

(9194, 6)

In [12]:
df1 = df1[df1.text.str.len()>100]
df1.shape

(9194, 6)

In [ ]:
filename = 'ohchr.csv'
sizes = []

df = pd.read_csv(filename, header=[0])

In [16]:
df1=df.copy()
sizes.append(df1.shape[0])

if 'url' not in df1.columns:
    df1['url'] ='\n'
if 'title' not in df1.columns:
    df1['title']='\n'

#     df1 = df1[~df1.url.isnull()]

df1 = df1[~df1.text.isnull()]
df1 = df1[~df1.text.duplicated()]
#     df1 = df1[~df1.url.duplicated()]

df1.loc[df1.title.isnull(),'title']='\n'
df1.loc[df1.url.isnull(),'url']='\n'
if 'abstract' in df1:
    df1.loc[df1.abstract.isnull(),'abstract']='\n'

# ------------------- Start cleaning  --------------------------#

# Convert coding
df1.text = df1.text.apply(lambda x: unicodetoascii(x))
df1.text = df1.text.apply(lambda x: unicodedata.normalize("NFKD", x))

# email
df1.text = df1.text.apply(lambda x: re.sub("\S+@\S+(?:\.\S+)+",'',x))

# telphone
df1.text = df1.text.apply(lambda x: re.sub('\(\+( |-|\d)+\)( |-|\d)+',' ',x))
df1.text = df1.text.apply(lambda x: re.sub('\+( |-|\d)+',' ',x))

In [22]:
df[df.url.duplicated()]

,title,text,country,url,country_url,news_search_link
49,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan","Human Rights Council\n\n12 March 2020\n\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan.\n\nNazhat S...",Botswana,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BWIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BW
55,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,Working Group to Review the Human Rights Records in 14 States\n\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council\'s Universal Periodic Review (UPR) Working Group w...,Botswana,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BWIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BW
69,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan","Human Rights Council\n\n12 March 2020\n\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan.\n\nNazhat S...",Burkina Faso,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BFIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BF
70,Angola’s human rights record to be reviewed by Universal Periodic Review,Portuguese version\n\nGENEVA (4 November 2019) - Angola's human rights record will be examined by the UN Human Rights Council's Universal Periodic Review (UPR) Working Group for the third time on ...,Burkina Faso,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BFIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BF
71,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,Working Group to Review the Human Rights Records in 14 States\n\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council\'s Universal Periodic Review (UPR) Working Group w...,Burkina Faso,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BFIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BF
...,...,...,...,...,...,...
9185,Press briefing notes on Israel / occupied Palestinian territory,Spokesperson for the UN High Commissioner for Human Rights: Ravina Shamdasani\n\nLocation: Geneva\n\nDate: 20 June 2014\n\nWe continue to follow closely the situation of Palestinian hunger striker...,Occupied Palestinian Territory,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14750&LangID=E,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS
9187,"Press briefing notes on Israel/OPT, Syria/Aleppo Central Prison and Dominican Republic/new citizenship law",Spokesperson for the UN High Commissioner for Human Rights: Cecile Pouilly\n\nLocation: Geneva\n\nDate: 23 May 2014\n\nSubject: 1) Israel/OPT\n\n2) Syria/Aleppo Central Prison\n\n3) Dominican Repu...,Occupied Palestinian Territory,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14632&LangID=E,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS
9189,"Press briefing notes on South Sudan, Ethiopia, United States, Palestine and Thailand / South East Asia",Spokesperson for the UN High Commissioner fo

In [20]:
df[df.text.duplicated()]

,title,text,country,url,country_url,news_search_link
49,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan","Human Rights Council\n\n12 March 2020\n\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan.\n\nNazhat S...",Botswana,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BWIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BW
55,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,Working Group to Review the Human Rights Records in 14 States\n\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council\'s Universal Periodic Review (UPR) Working Group w...,Botswana,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BWIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BW
69,"Human Rights Council adopts Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan","Human Rights Council\n\n12 March 2020\n\nThe Human Rights Council in its midday meeting adopted the Universal Periodic Review outcomes of Fiji, San Marino, Iran, Angola and Kazakhstan.\n\nNazhat S...",Burkina Faso,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25717&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BFIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BF
70,Angola’s human rights record to be reviewed by Universal Periodic Review,Portuguese version\n\nGENEVA (4 November 2019) - Angola's human rights record will be examined by the UN Human Rights Council's Universal Periodic Review (UPR) Working Group for the third time on ...,Burkina Faso,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25251&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BFIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BF
71,Human Rights Council Universal Periodic Review Working Group to hold thirty-fourth session in Geneva from 4 to 15 November 2019,Working Group to Review the Human Rights Records in 14 States\n\nGENEVA (30 October 2019) - The thirty-fourth session of the Human Rights Council\'s Universal Periodic Review (UPR) Working Group w...,Burkina Faso,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=25228&LangID=E,https://www.ohchr.org/EN/countries/AfricaRegion/Pages/BFIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=BF
...,...,...,...,...,...,...
9185,Press briefing notes on Israel / occupied Palestinian territory,Spokesperson for the UN High Commissioner for Human Rights: Ravina Shamdasani\n\nLocation: Geneva\n\nDate: 20 June 2014\n\nWe continue to follow closely the situation of Palestinian hunger striker...,Occupied Palestinian Territory,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14750&LangID=E,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS
9187,"Press briefing notes on Israel/OPT, Syria/Aleppo Central Prison and Dominican Republic/new citizenship law",Spokesperson for the UN High Commissioner for Human Rights: Cecile Pouilly\n\nLocation: Geneva\n\nDate: 23 May 2014\n\nSubject: 1) Israel/OPT\n\n2) Syria/Aleppo Central Prison\n\n3) Dominican Repu...,Occupied Palestinian Territory,https://www.ohchr.org/en/NewsEvents/Pages/DisplayNews.aspx?NewsID=14632&LangID=E,https://www.ohchr.org/EN/countries/MENARegion/Pages/PSIndex.aspx,https://www.ohchr.org/en/NewsEvents/Pages/NewsSearch.aspx?CID=PS
9189,"Press briefing notes on South Sudan, Ethiopia, United States, Palestine and Thailand / South East Asia",Spokesperson for the UN High Commissioner fo

In [18]:
df.text.str.len().describe()

count     9194.000000
mean     11110.418751
std      12778.448149
min        449.000000
25%       2897.750000
50%       4981.000000
75%      14392.250000
max      99288.000000
Name: text, dtype: float64

In [19]:
df1.text.str.len().describe()

count     4876.000000
mean     10839.633306
std      12761.770125
min        449.000000
25%       3372.000000
50%       4853.000000
75%      11515.750000
max      99288.000000
Name: text, dtype: float64

In [13]:

# noise
df1.text =\
df1.text.apply(lambda x: re.sub('\n(ad|advertisement|tweet):?\n', "", x, flags=re.IGNORECASE))

# urls
df1.text = df1.text.apply(lambda x: re.sub(r"http\S+", "", x))

# delete too many \n
df1.text = df1.text.apply(lambda x: re.sub('\n\n+', "\n\n", x, flags=re.IGNORECASE))

# head and tails
df1.text = df1.text.apply(lambda x: re.sub("^\s+|\s+$", "", x, flags=re.UNICODE)) 


df1 = df1[df1.text.str.len()>100]

sizes.append(df1.shape[0])

# ------------------- Ending cleaning  --------------------------#

# filename = filename.split('/')[1]
new_filename = output_folder + '/'+ filename
print(new_filename)

IndexError: list index out of range

In [15]:
sizes

[9194, 4876]

In [14]:
# filename = filename.split('/')[1]
# new_filename = output_folder + '/'+ filename
# print(new_filename)

NameError: name 'output_folder' is not defined